In [8]:
import numpy as np
import pandas as pd
import pickle
from collections import Counter

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import wordpunct_tokenize

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import os
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [58]:
DATA_PATH = "./scrapped_data/"
TRAIN_PATH = os.path.join(DATA_PATH, "scrapped_train_emb.csv")
TEST_PATH = os.path.join(DATA_PATH, "scrapped_test_emb.csv")
MEAN_TARGET_ENCODING_PATH = "./mean_target_encoding"
TEST_RATIO = 0.2

In [44]:
data = pd.read_csv(TRAIN_PATH, index_col=0)
data

,title,publication,author,followers,reading_time,n_words,pure_text,date,n_code_chunks,bold_text_count,...,nymmurb,ocoaoxx,yvhavwm,cefjhoa,kjymdpz,jylkomw,rncigks,oqcbsyc,pfzwata,ffxdjat
idx,,,,,,,,,,,,,,,,,,,,,
0,Understanding Variational Autoencoders (VAEs),Towards Data Science,Joseph Rocca,3700.0,23.0,4892.0,This post was co-written with In the last few ...,24/09/2019,0.0,35.0,...,-0.000058,0.001408,-0.000032,0.001562,0.001159,0.000113,-0.001006,-0.001673,-0.000658,-0.001348
1,Using Mixed-Effects Models For Linear Regression,Towards Data Science,Guido Vivaldi,256.0,6.0,889.0,Mixed-effects regression models are a powerful...,18/05/2019,9.0,6.0,...,0.001289,0.000647,0.000037,-0.000059,0.001223,0.001932,0.001527,-0.001245,0.002017,0.000744
2,The New Moats,Greylock Perspectives,Jerry Chen,4700.0,11.0,2217.0,To build a sustainable and profitable business...,24/04/2017,0.0,5.0,...,0.000771,0.001435,0.002123,0.001547,-0.000377,0.001728,-0.000275,-0.001748,-0.001030,-0.001139
3,Region of Interest Pooling,Towards Data Science,Sambasivarao. K,239.0,4.0,523.0,The major hurdle for going from image classifi...,22/04/2019,4.0,5.0,...,-0.000745,0.003573,0.000464,0.001312,0.003607,0.002550,0.001222,-0.001847,0.002282,-0.001934
4,Modern Gaussian Process Regression,Towards Data Science,Ryan Sander,173.0,10.0,1433.0,Ever wonder how you can create non-parametric ...,24/03/2021,2.0,96.0,...,0.000716,0.003237,0.002098,0.000814,-0.000241,0.002537,-0.001218,0.000696,0.001832,-0.000393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3751,Nerves: “Gold Month of Exchanges” — listed on ...,Nerves Foundation,Nerves Foundation,1800.0,1.0,153.0,"Dear Nervesians,Today is probably a special da...",21/09/2018,0.0,9.0,...,0.002017,0.008279,0.006440,0.000711,0.006402,0.010626,0.001607,-0.004667,0.005139,-0.000928
3752,An introduction to ConvLSTM,Neuronio,Alexandre Xavier,124.0,6.0,1009.0,Note: a Portuguese version of this article is ...,25/03/2019,0.0,21.0,...,0.001265,0.001075,0.002265,-0.000864,-0.000121,0.001508,0.001126,-0.001947,0.000481,-0.001287
3753,Automatic Text Summarization Made Simple with ...,luisfredgs,Luís Gonçalves,317.0,4.0,632.0,Summarization condenses a longer document into...,19/04/2020,7.0,1.0,...,0.000750,0.002869,0.001904,0.000846,0.003289,0.002529,0.001014,-0.001781,0.001972,-0.001122


In [11]:
data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3756 entries, 0 to 3755
Data columns (total 332 columns):
 #    Column             Non-Null Count  Dtype  
---   ------             --------------  -----  
 0    title              3756 non-null   object 
 1    publication        3756 non-null   object 
 2    author             3756 non-null   object 
 3    followers          3756 non-null   float64
 4    reading_time       3756 non-null   float64
 5    n_words            3756 non-null   float64
 6    pure_text          3756 non-null   object 
 7    date               3756 non-null   object 
 8    n_code_chunks      3756 non-null   float64
 9    bold_text_count    3756 non-null   float64
 10   italic_text_count  3756 non-null   float64
 11   mean_image_width   3756 non-null   float64
 12   mean_image_height  3756 non-null   float64
 13   n_images           3756 non-null   float64
 14   n_lists            3756 non-null   float64
 15   n_vids             3756 non-null   float64
 16   n_li

In [12]:
data["language"].value_counts()

en       3507
es         25
ko         20
th         20
tr         19
zh-tw      17
fr         17
it         17
id         14
de         13
ru         12
ca         11
vi         11
pt          8
tl          6
nl          6
no          6
zh-cn       4
ja          4
af          4
sw          3
da          3
cy          2
sv          2
ro          2
fi          1
pl          1
lv          1
Name: language, dtype: int64

In [55]:
def estimate_mean_target_regularization(category, global_mean: float, category_means: pd.Series,
                                        category_counts: pd.Series, alpha: float = 10) -> float:
    try:
        return (category_counts[category] * category_means[category] + global_mean * alpha) / (category_counts[category] * alpha)
    except:
        return np.nan

In [56]:
def mean_target_regularization(dataset: pd.DataFrame, column: str, target_name: str,
                               train: bool = True, alpha: float = 10) -> pd.Series:
    encoding_file_path = MEAN_TARGET_ENCODING_PATH + "_" + column + ".pickle"
    if train:
        encoding = {
            "global_mean": dataset[target_name].mean(),
            "counts": dataset.groupby(dataset[column]).count()[target_name],
            "category_means": dataset.groupby(dataset[column]).mean()[target_name]
        }
        with open(encoding_file_path, "wb") as output_file:
            pickle.dump(encoding, output_file)
    else:
        with open(encoding_file_path, "rb") as input_file:
            encoding = pickle.load(input_file)
    
    global_mean = encoding["global_mean"]
    counts = encoding["counts"]
    category_means = encoding["category_means"]
    
    target_mean = dataset[column].apply(
        lambda category: estimate_mean_target_regularization(
            category,
            global_mean=global_mean,
            category_means=category_means,
            category_counts=counts,
            alpha=alpha
        )
    )
    
    return target_mean

In [59]:
def preprocess_data(data: pd.DataFrame, train: bool = True):
    result_data = data.copy()
    
    # Mean target encoding with regularization
    # result_data["author_mean_target"] = mean_target_regularization(result_data, "author", "target", train=train)
    # result_data["publication_mean_target"] = mean_target_regularization(result_data, "publication", "target", train=train)
    
    
    # Add title len as a feature
    result_data["title_char_len"] = result_data["title"].apply(len)
    result_data["title_token_len"] = result_data["title"].apply(lambda title: len(wordpunct_tokenize(title)))
    
    # Encode dates
    dates = result_data["date"].apply(lambda date: pd.Timestamp(date))
    result_data["day"] = dates.dt.day
    result_data["month"] = dates.dt.month
    result_data["year"] = dates.dt.year
    
    # Season encoding
    season_dict = {
        1: 'winter',
        2: 'winter',
        3: 'spring',
        4: 'spring',
        5: 'spring',
        6: 'summer',
        7: 'summer',
        8: 'summer',
        9: 'fall',
        10: 'fall',
        11: 'fall',
        12: 'winter'
    }
    result_data['season'] = result_data['month'].map(lambda x: season_dict[x])
    result_data['winter'] = [1 if season=='winter' else 0 for season in result_data['season']]
    result_data['summer'] = [1 if season=='summer' else 0 for season in result_data['season']]
    result_data['spring'] = [1 if season=='spring' else 0 for season in result_data['season']]
    result_data['fall'] = [1 if season=='fall' else 0 for season in result_data['season']]
    
    keyword_columns = [f"key_word_{idx}" for idx in range(1, 11)]

    # Filling nans in keyword_1, ..., keyword_10 columns
    for keyword_column in keyword_columns:
        result_data[keyword_column].fillna("<unk>", inplace=True)
        
    assert result_data[keyword_columns].isna().sum().sum() == 0
    
    # Count topics for keyword_columns
    MOST_COMMON_COUNT = 50
    lemmatizer = WordNetLemmatizer()
    result_data[keyword_columns] = result_data[keyword_columns].applymap(lemmatizer.lemmatize)
    
    topics = []
    for keyword_column in keyword_columns:
        topics += result_data[keyword_column].tolist()
    
    topics_counter = Counter(topics)
    most_common_topics = [topic for topic, _ in topics_counter.most_common()[:MOST_COMMON_COUNT]]
    
    # Mark less frequent topics as <unk>
    result_data[keyword_columns][~result_data[keyword_columns].isin(most_common_topics)] = "<unk>"
    
    assert result_data[keyword_columns].isin(most_common_topics).sum().sum() > 0

    assert len(most_common_topics) == MOST_COMMON_COUNT
    
    result_data['is_english'] = np.where((result_data.language == 'en'), 1, 0)

    drop_columns = ["date", "author", "publication", "season", "pure_text", "title", 'language'] + keyword_columns
    result_data.drop(drop_columns, inplace=True, axis=1)
    return pd.get_dummies(result_data)

In [61]:
submission = preprocess_data(pd.read_csv(TEST_PATH, index_col='idx'))

C:\Users\Aleksey\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\series.py:4430: UserWarning: Parsing '18/10/2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
C:\Users\Aleksey\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\series.py:4430: UserWarning: Parsing '16/01/2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
C:\Users\Aleksey\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\series.py:4430: UserWarning: Parsing '21/11/2005' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
C:\Users\Aleksey\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core

In [62]:
data_preprocessed = preprocess_data(data)
data_preprocessed

C:\Users\Aleksey\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\series.py:4430: UserWarning: Parsing '24/09/2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
C:\Users\Aleksey\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\series.py:4430: UserWarning: Parsing '18/05/2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
C:\Users\Aleksey\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\series.py:4430: UserWarning: Parsing '24/04/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
C:\Users\Aleksey\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core

,followers,reading_time,n_words,n_code_chunks,bold_text_count,italic_text_count,mean_image_width,mean_image_height,n_images,n_lists,...,analytic_subjective,polarity_neutral,polarity_positive,emotion_anger/irritation,emotion_contempt/reluctance,emotion_expectation/interest,emotion_joy/calmness,emotion_no emotion,emotion_surprise/amazement,emotion_trust/acceptance
idx,,,,,,,,,,,,,,,,,,,,,
0,3700.0,23.0,4892.0,0.0,35.0,1.0,1150.0,426.0,30.0,3.0,...,0,1,0,0,0,0,1,0,0,0
1,256.0,6.0,889.0,9.0,6.0,1.0,516.0,304.0,13.0,1.0,...,0,1,0,0,0,0,1,0,0,0
2,4700.0,11.0,2217.0,0.0,5.0,27.0,994.0,582.0,6.0,2.0,...,0,1,0,0,0,1,0,0,0,0
3,239.0,4.0,523.0,4.0,5.0,0.0,753.0,512.0,2.0,2.0,...,0,1,0,0,0,0,1,0,0,0
4,173.0,10.0,1433.0,2.0,96.0,15.0,657.0,291.0,14.0,4.0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3751,1800.0,1.0,153.0,0.0,9.0,28.0,1400.0,716.0,1.0,0.0,...,0,1,0,0,0,1,0,0,0,0
3752,124.0,6.0,1009.0,0.0,21.0,0.0,859.0,631.0,4.0,3.0,...,0,1,0,0,0,0,0,0,1,0
3753,317.0,4.0,632.0,7.0,1.0,2.0,700.0,405.0,1.0,1.0,...,0,1,0,0,0,0,1,0,0,0


In [67]:
len(set(submission.columns) - set(data_preprocessed.columns)) == 0

True

In [68]:
X = data_preprocessed.drop(["target"], axis=1).to_numpy()
y = data_preprocessed["target"].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_RATIO, shuffle=True)

In [69]:
cb_regressor = CatBoostRegressor(verbose=True)
cb_regressor.fit(X_train, y_train)

cb_predictions = cb_regressor.predict(X_test)
cb_predictions[cb_predictions < 0] = 0
cb_predictions = np.round(cb_predictions)
mean_absolute_error(y_test, cb_predictions)

Learning rate set to 0.048714
0:	learn: 4172.2004045	total: 78.5ms	remaining: 1m 18s
1:	learn: 4124.2946079	total: 135ms	remaining: 1m 7s
2:	learn: 4075.2246173	total: 188ms	remaining: 1m 2s
3:	learn: 4024.2820477	total: 246ms	remaining: 1m 1s
4:	learn: 3967.2576226	total: 303ms	remaining: 1m
5:	learn: 3920.4644387	total: 360ms	remaining: 59.6s
6:	learn: 3867.8116753	total: 420ms	remaining: 59.5s
7:	learn: 3827.0915046	total: 481ms	remaining: 59.6s
8:	learn: 3783.9907996	total: 555ms	remaining: 1m 1s
9:	learn: 3738.8706734	total: 618ms	remaining: 1m 1s
10:	learn: 3698.6934926	total: 685ms	remaining: 1m 1s
11:	learn: 3656.0809492	total: 749ms	remaining: 1m 1s
12:	learn: 3622.9287157	total: 818ms	remaining: 1m 2s
13:	learn: 3578.4162536	total: 878ms	remaining: 1m 1s
14:	learn: 3542.6770572	total: 932ms	remaining: 1m 1s
15:	learn: 3506.7559324	total: 987ms	remaining: 1m
16:	learn: 3479.7167752	total: 1.05s	remaining: 1m
17:	learn: 3433.7439898	total: 1.11s	remaining: 1m
18:	learn: 3407.46

1127.3696808510638

In [70]:
lgbm_regressor = LGBMRegressor()
lgbm_regressor.fit(X_train, y_train)

lgbm_predictions = lgbm_regressor.predict(X_test)
lgbm_predictions[lgbm_predictions < 0] = 0
lgbm_predictions = np.round(lgbm_predictions)
mean_absolute_error(y_test, lgbm_predictions)

1292.3178191489362

In [71]:
xgb_regressor = XGBRegressor()
xgb_regressor.fit(X_train, y_train)

xgb_predictions = xgb_regressor.predict(X_test)
xgb_predictions[xgb_predictions < 0] = 0
xgb_predictions = np.round(xgb_predictions)
mean_absolute_error(y_test, xgb_predictions)

1127.8218085106382